In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import wandb
from wandb.keras import WandbMetricsLogger
import sys
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from save_weights_every_epoch import CallbackForSavingModelWeights

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size)
    train, val = train_test_split(train, test_size=test_size)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [8]:
def read_train_imgs(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.resize(img, (512, 512))
    img = img / 255
    return img, label

def read_imgs(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, (512, 512))
    img = img / 255
    return img, label

In [9]:
def get_data(csv_file, encoder, is_train = True, repeat = True, batch=True, shuffle=True, batch_size=32):
    imgs, labels = csv_file['updated_paths'].values.tolist(), csv_file['species'].values.tolist()
    labels = encoder.transform(np.array(labels).reshape(-1, 1)).toarray()
    tensor = tf.data.Dataset.from_tensor_slices((imgs, labels))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    if is_train:
        tensor = tensor.map(read_train_imgs, num_parallel_calls=AUTO)
    else:
        tensor = tensor.map(read_imgs, num_parallel_calls=AUTO)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [10]:
def get_model():
    input_layer = keras.layers.Input((512, 512, 3))
    model_layer = keras.applications.InceptionV3(
        include_top=False,
        weights = None,
        input_shape=(512, 512, 3),
        pooling = 'avg'
    )(input_layer)
    last_layer = keras.layers.Dense(30, activation='softmax')(model_layer)
    model = tf.keras.Model(
        input_layer,
        last_layer,
        name = 'IncepModel'
    )
    # model.summary()
    return model

In [11]:
def compile_new_model():
    with strategy.scope():
        model = get_model()
        losses = keras.losses.CategoricalCrossentropy(),
        metrics = [
            keras.metrics.Precision(name='prec'),
            keras.metrics.Recall(name='rec')
        ]
        model.compile(
            optimizer = keras.optimizers.SGD(),
            loss = losses,
            metrics = metrics
        )
    return model

In [12]:
# get_data(csv_file, encoder, repeat = True, batch=True, shuffle=True, batch_size=32):

In [18]:
path_to_save = '../../tf_weights/incep_v1/'
save_weights = CallbackForSavingModelWeights(path_to_save)
batch_size = 176
train_dataset = get_data(train, encoder, batch_size = batch_size)
val_dataset = get_data(val, encoder, shuffle = False, repeat = False, batch_size = batch_size)
wandb.init(
    project='dolphin',
    config = {
        'batch_size' : batch_size,
    }
)
model = compile_new_model()
hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    steps_per_epoch = len(train) // batch_size,
    epochs = 50,
    verbose = 1,
    callbacks = [
        WandbMetricsLogger(),
        save_weights
    ]
)

epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▄▃▂▁
epoch/prec,▁▄▆▇█
epoch/rec,▁▄▆▇█
epoch/val_loss,█▄▂▁▃
epoch/val_prec,▁▇██▆
epoch/val_rec,▁▄▇█▆
epoch/epoch,4
epoch/learning_rate,0.01
epoch/loss,0.72959


Epoch 1/50
284/284 [==============================] - 199s 636ms/step - loss: 1.9848 - prec: 0.7761 - rec: 0.1941 - val_loss: 3.9685 - val_prec: 0.0556 - val_rec: 0.0020
Epoch 2/50
284/284 [==============================] - 180s 634ms/step - loss: 1.2738 - prec: 0.8332 - rec: 0.4653 - val_loss: 1.6957 - val_prec: 0.7677 - val_rec: 0.3854
Epoch 3/50
284/284 [==============================] - 180s 634ms/step - loss: 0.9781 - prec: 0.8625 - rec: 0.5820 - val_loss: 1.3917 - val_prec: 0.7515 - val_rec: 0.5079
Epoch 4/50
284/284 [==============================] - 179s 631ms/step - loss: 0.7978 - prec: 0.8838 - rec: 0.6597 - val_loss: 1.4626 - val_prec: 0.7316 - val_rec: 0.4526
Epoch 5/50
284/284 [==============================] - 180s 633ms/step - loss: 0.6620 - prec: 0.8961 - rec: 0.7169 - val_loss: 0.9016 - val_prec: 0.8260 - val_rec: 0.6660
Epoch 6/50
284/284 [==============================] - 180s 634ms/step - loss: 0.5570 - prec: 0.9110 - rec: 0.7638 - val_loss: 0.9599 - val_prec: 0.797

KeyboardInterrupt: 

In [12]:
path_to_save = '../../tf_weights/incep_v1_flip_iamges/'
save_weights = CallbackForSavingModelWeights(path_to_save)
batch_size = 176
train_dataset = get_data(train, encoder, is_train = True, batch_size = batch_size)
val_dataset = get_data(val, encoder, is_train = False, shuffle = False, repeat = False, batch_size = batch_size)
wandb.init(
    project='dolphin',
    config = {
        'batch_size' : batch_size,
        'flip_left_right' : True,
    }
)
model = compile_new_model()
hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    steps_per_epoch = len(train) // batch_size,
    epochs = 50,
    verbose = 1,
    callbacks = [
        WandbMetricsLogger(),
        save_weights
    ]
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu